In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

C:\Users\baek2\AppData\Local\Temp\ipykernel_37452\804421021.py:3: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [2]:
# ES 클라이언트 정의
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

load_dotenv()

url = os.getenv("ELASTIC_CLOUD_URL")
api_id = os.getenv("ELASTIC_API_ID")
api_key = os.getenv("ELASTIC_API_KEY")
es_model_id = os.getenv("ELASTIC_MODEL_ID")


client = Elasticsearch(
    url,
    api_key=(api_id, api_key)
)

print(client.info())

{'name': 'instance-0000000000', 'cluster_name': 'a30a7c8ef0bf435a9d350006622225d8', 'cluster_uuid': 'Y2Ofhp1kTYe--9mTVeNzVQ', 'version': {'number': '9.2.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '4ad0ef0e98a2e72fafbd79a19fa5cae2f026117d', 'build_date': '2025-11-06T22:07:39.673130621Z', 'build_snapshot': False, 'lucene_version': '10.3.1', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [3]:
import os
import glob
from typing import List, Dict

import pandas as pd
from PyPDF2 import PdfReader

# 1. DATA > CSV

In [4]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    단일 PDF 파일에서 모든 페이지 텍스트를 추출하는 함수.
    """
    reader = PdfReader(pdf_path)
    texts = []

    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            texts.append(page_text)

    return "\n".join(texts)


In [5]:
def extract_text_from_txt(txt_path: str) -> str:
    """
    단일 텍스트(.txt) 파일에서 텍스트를 읽어오는 함수.
    """
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        # utf-8로 읽기 실패 시, 한글 윈도우 기본 인코딩인 cp949로 시도
        try:
            with open(txt_path, 'r', encoding='cp949') as f:
                return f.read()
        except Exception as e:
            print(f"Error reading {txt_path}: {e}")
            return ""

In [6]:

def chunk_text(
    text: str,
    max_chars: int = 1000,
    overlap: int = 200
) -> List[str]:
    """
    긴 텍스트를 max_chars 기준으로 잘라 chunk 리스트를 반환.
    overlap만큼 앞 chunk와 겹치게 슬라이딩 윈도우 형태로 자름.
    """
    text = " ".join(text.split())
    
    if not text:
        return []

    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk)

        start = end - overlap
        
        if start < 0:
            start = 0
        
        if start >= text_length:
            break
            

    return chunks

In [ ]:
def build_document_chunk_dataframe(
    folder_path: str,
    max_chars: int = 1000,
    overlap: int = 200
) -> pd.DataFrame:
    """
    주어진 폴더에서 PDF와 TXT 파일을 읽고 DataFrame 생성.
    """
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))
    all_files = pdf_files + txt_files
    
    records: List[Dict] = []

    for file_path in all_files:
        filename = os.path.basename(file_path)
        _, ext = os.path.splitext(filename)
        ext = ext.lower()
        
        print(f"Processing: {filename}")

        full_text = ""
        if ext == '.pdf':
            full_text = extract_text_from_pdf(file_path)
        elif ext == '.txt':
            full_text = extract_text_from_txt(file_path)
        
        if not full_text.strip():
            continue

        chunks = chunk_text(full_text, max_chars=max_chars, overlap=overlap)

        for i, chunk in enumerate(chunks, start=1):
            # chunk_text 맨 앞에 파일명을 명시적으로 추가
            chunk_with_filename = f"[{filename}] {chunk}"

            records.append(
                {
                    "filename": filename,
                    "extension": ext,
                    "chunk_seq": i,
                    "chunk_text": chunk_with_filename, # 파일명이 포함된 텍스트 저장
                }
            )

    df = pd.DataFrame(records, columns=["filename", "extension", "chunk_seq", "chunk_text"])
    return df


In [8]:
# 1.1 폴더 내 PDF파일을 읽어서 데이터 프레임으로 저장

folder_path = "D:/workspace/대학원/25년도2학기/정보검색프로젝트/색인데이터"

max_chars = 1000
overlap = 200

df = build_document_chunk_dataframe(
    folder_path=folder_path,
    max_chars=max_chars,
    overlap=overlap
    )

print(df.head())

Processing: 01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf
Processing: 02.3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf
Processing: 03.4주차 ElasticSearch.pdf
Processing: 04.5주차 LLM 이해와 PromptEngineering 9월 30일.pdf
Processing: 05.7주차 생성형AI 검색 기초 - 벡터검색과 RAG_1014.pdf
Processing: 06-1.LangChain 기초_별첨.pdf
Processing: 06.제10강 검색에이전트와 LangChain LangGraph.pdf
Processing: 07.제11강 LangChain LangGraph를 이용한 AgenticRAG 개발.pdf
Processing: 08.제12강 Multi-Hop .pdf
Processing: 09.제13강 멀티모달 검색.pdf
Processing: 07-1.11강 LangChain LangGraph를 이용한 AgenticRag개발_11월11일수업내용.txt
Processing: 09-1.13강 멀티모달 검색_1125일수업내용.txt
                               filename extension  chunk_seq  \
0  01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf      .pdf          1   
1  01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf      .pdf          2   
2  01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf      .pdf          3   
3  01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf      .pdf          4   
4  01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf      .pdf          5   

                                   

In [9]:
# 1.2 데이터 프레임을 CSV로 저장

df.to_csv("pdf_chunks.csv", index=False, encoding="utf-8-sig")

In [10]:

class_info = pd.read_csv('./pdf_chunks.csv', encoding='utf-8-sig')
class_info.loc[1:2]

,filename,extension,chunk_seq,chunk_text
1,01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf,.pdf,2,[01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf] (Vector...
2,01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf,.pdf,3,[01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf] 관련(?) ...


In [ ]:
class_info

,filename,extension,chunk_seq,chunk_text
0,01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf,.pdf,1,[01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf] 빅데이터 정보...
1,01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf,.pdf,2,[01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf] (Vector...
2,01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf,.pdf,3,[01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf] 관련(?) ...
3,01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf,.pdf,4,[01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf] 등 의미없는...
4,01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf,.pdf,5,[01.2주차 검색개요 - 색인과 검색랭킹 모델(part1).pdf] 는 검색키워드...
...,...,...,...,...
345,09-1.13강 멀티모달 검색_1125일수업내용.txt,.txt,34,[09-1.13강 멀티모달 검색_1125일수업내용.txt] 해서 그와 관련된 뭘 찾...
346,09-1.13강 멀티모달 검색_1125일수업내용.txt,.txt,35,[09-1.13강 멀티모달 검색_1125일수업내용.txt] 는 우리 벤치마크 데이터...
347,09-1.13강 멀티모달 검색_1125일수업내용.txt,.txt,36,[09-1.13강 멀티모달 검색_1125일수업내용.txt] 에서 수행할 수 있도록 ...
348,09-1.13강 멀티모달 검색_1125일수업내용.txt,.txt,37,[09-1.13강 멀티모달 검색_1125일수업내용.txt] 하는 거예요. 근데 여...


# 2. ES 색인 시작

In [14]:
index_name = "class-info"

In [3]:
# 2.1 임베딩을 위한 ingest pipeline 추가 

client.ingest.put_pipeline(
    id="pipeline",
    processors=[
        {
            "inference": {
                # embedding에 활용할 model_id 지정
                "model_id": es_model_id,  

                # embedding 대상 text를 chunk_text 필드로 지정
                "field_map": {"chunk_text": "text_field"},
                  
                # embedding 결과를 chunk_embedding 필드에 저장
                "target_field": "chunk_embedding", 
            }
        }
    ],
)

ObjectApiResponse({'acknowledged': True})

In [ ]:
# 2.2 색인을 위한 analyzer, mapping 구성, pipeline 연결 

index_body = {
    "settings": {
        "index.mapping.exclude_source_vectors": False, 
        "analysis": {
            "tokenizer": {
                "nori_tokenizer_custom": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "mixed"
                }
            },
            "analyzer": {
                "korean_nori": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer_custom",
                    "filter": [
                        "lowercase"
                    ]
                }
            }
        },
        "index": {
        "number_of_replicas": "1",
        "number_of_shards": "1",
        "default_pipeline": "pipeline",          # 파이프라인 적용설정
        }
    },
    "mappings": {
        "properties": {
            "filename": {
                "type": "keyword"
            },
            "chunk_seq": {
                "type": "integer"
            },
            "chunk_text": {                      # SparseSearch 적용을 위한 설정
                "type": "text",                 
                "analyzer": "korean_nori"  
            },
            "chunk_embedding.predicted_value": { # DenseSearch 적용을 위한 설정
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

In [17]:
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)
    print(f"Deleted existing index: {index_name}")

In [18]:
# 2.3 인덱스 생성
response = client.indices.create(
    index=index_name,
    body=index_body
)

In [19]:
# 2.4 데이터 색인

indexed_results = []

for index, row in class_info.iterrows():
    doc_source = {
        "filename": row['filename'] if pd.notna(row['filename']) else None,
        "chunk_seq": row['chunk_seq'] if pd.notna(row['chunk_seq']) else None,
        "chunk_text": row['chunk_text'] if pd.notna(row['chunk_text']) else None
    }
    
    try:
        response = client.index(index="class-info", document=doc_source, id=str(index))
        indexed_results.append({'index': index, 'status': 'success', 'response': response})
        if index % 30 == 0:
            print(f"Successfully indexed document with index {index}")
    except Exception as e:
        indexed_results.append({'index': index, 'status': 'failed', 'error': str(e)})
        print(f"Failed to index document with index {index}: {e}")


Successfully indexed document with index 0
Successfully indexed document with index 30
Successfully indexed document with index 60
Successfully indexed document with index 90
Successfully indexed document with index 120
Successfully indexed document with index 150
Successfully indexed document with index 180
Successfully indexed document with index 210
Successfully indexed document with index 240
Successfully indexed document with index 270
Successfully indexed document with index 300
Successfully indexed document with index 330
